In [1]:
#加载所需要的库
!pip install transformers datasets evaluate tokenizer sacrebleu sentencepiece accelerate sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 68.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5

In [2]:
import numpy as np
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer,AutoConfig
import torch
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer,DataCollatorForSeq2Seq
#checkpoint = 'Helsinki-NLP//opus-mt-en-zh'
checkpoint = "Jyshen/Translation_en2zh"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [3]:
# Training for DDDSSS dataset
from datasets import Dataset
DS_dataset = load_dataset("DDDSSS/en-zh-dataset")
def DS_pfunc(examples):
    inputs = [example["en"] for example in examples["translation"]]
    targets = [example["zh"] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs
model_inputs = DS_dataset.map(DS_pfunc, batched=True)
model_inputs = Dataset.from_dict({'input_ids':model_inputs['train']['input_ids'],'attention_mask':model_inputs['train']['attention_mask'],'labels':model_inputs['train']['labels']})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/DDDSSS___json/DDDSSS--en-zh-dataset-44c5559b2e11d4bb/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/96538 [00:00<?, ? examples/s]

In [5]:
# Training for CoQA dataset
from datasets import concatenate_datasets
from huggingface_hub import notebook_login
notebook_login()

CoQA_dataset = load_dataset("silk-road/Luotuo-QA-A-CoQA-Chinese")
def Co_pfunc(examples):
    inputs = [example for example in examples["story"]]
    targets = [example for example in examples["story_zh"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs
model_inputs2 = CoQA_dataset.map(Co_pfunc, batched=True)
model_inputs2 = Dataset.from_dict({'input_ids':model_inputs2['train']['input_ids'],'attention_mask':model_inputs2['train']['attention_mask'],'labels':model_inputs2['train']['labels']})
model_inputs = concatenate_datasets([model_inputs, model_inputs2])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/silk-road___json/silk-road--Luotuo-QA-A-CoQA-Chinese-c3760ccb13c14226/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/7012 [00:00<?, ? examples/s]

In [6]:
# Training for Dolly dataset
Dolly_dataset = load_dataset("silk-road/chinese-dolly-15k")

def Do_pfunc(examples):
    inputs = [example for example in examples["instruction"]]
    targets = [example for example in examples["instruction_zh"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs
model_inputs2 = Dolly_dataset.map(Do_pfunc, batched=True)
model_inputs2 = Dataset.from_dict({'input_ids':model_inputs2['train']['input_ids'],'attention_mask':model_inputs2['train']['attention_mask'],'labels':model_inputs2['train']['labels']})
model_inputs = concatenate_datasets([model_inputs, model_inputs2])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/silk-road___json/silk-road--chinese-dolly-15k-a9aca306f7816495/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

In [7]:
# Training for MMC4 dataset
MMC4_dataset = load_dataset("silk-road/MMC4-130k-chinese-image")

def M4_pfunc(examples):
    inputs = [example for example in examples["caption"]]
    targets = [example for example in examples["caption_zh"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs
model_inputs2 = MMC4_dataset.map(M4_pfunc, batched=True)
model_inputs2 = Dataset.from_dict({'input_ids':model_inputs2['train']['input_ids'],'attention_mask':model_inputs2['train']['attention_mask'],'labels':model_inputs2['train']['labels']})
model_inputs = concatenate_datasets([model_inputs, model_inputs2])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/silk-road___json/silk-road--MMC4-130k-chinese-image-e5babaac954775f5/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/135463 [00:00<?, ? examples/s]

In [8]:
model_inputs = model_inputs.train_test_split(test_size= 0.2,shuffle= True,seed= 2023)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)
metric = evaluate.load("sacrebleu")

In [9]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
batchsize=8
training_args = Seq2SeqTrainingArguments(
    output_dir="./best_model",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=batchsize,
    per_device_eval_batch_size=batchsize,
    weight_decay=0.01,
    # save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    save_strategy="epoch",
    jit_mode_eval=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=model_inputs["train"],
    eval_dataset=model_inputs["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/modeling_marian.py:242: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_weights.size() != (bsz * self.num_heads, tgt_len, src_len):
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/modeling_marian.py:249: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attention_mask.size() != (bsz, 1, tgt_len, src_len):
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/modeling_marian.py:281: TracerWarning: Converting a tensor to a Python boolean might cause th

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.851200,1.656123,14.020200,34.262800
2,1.236100,1.296612,17.932600,34.595100


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/modeling_marian.py:242: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_weights.size() != (bsz * self.num_heads, tgt_len, src_len):
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/modeling_marian.py:249: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attention_mask.size() != (bsz, 1, tgt_len, src_len):
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/modeling_marian.py:281: TracerWarning: Converting a tensor to a Python boolean might cause th

TrainOutput(global_step=50806, training_loss=1.8351851413274654, metrics={'train_runtime': 27143.5938, 'train_samples_per_second': 14.974, 'train_steps_per_second': 1.872, 'total_flos': 1.2753134272512e+16, 'train_loss': 1.8351851413274654, 'epoch': 2.0})

In [10]:
dir = "/content/drive/MyDrive/best_model/"
model.save_pretrained(dir)
tokenizer.save_pretrained(dir)

('/content/drive/MyDrive/best_model/tokenizer_config.json',
 '/content/drive/MyDrive/best_model/special_tokens_map.json',
 '/content/drive/MyDrive/best_model/vocab.json',
 '/content/drive/MyDrive/best_model/source.spm',
 '/content/drive/MyDrive/best_model/target.spm',
 '/content/drive/MyDrive/best_model/added_tokens.json')